#  6.2 中文文本处理

## 6.2.1 中文分词

In [3]:
#例子6-1：将文本句子 “2018年世界杯小组赛抽签在莫斯科克里姆林宫举行”进行分词。

import jieba
print( jieba.lcut("2018年世界杯小组赛抽签在莫斯科克里姆林宫举行") )
print( jieba.lcut("2018年世界杯小组赛抽签在莫斯科克里姆林宫举行", cut_all = True) )
print( jieba.lcut_for_search("2018年世界杯小组赛抽签在莫斯科克里姆林宫举行") )

['2018', '年', '世界杯', '小组赛', '抽签', '在', '莫斯科', '克里姆林宫', '举行']
['2018', '年', '世界', '世界杯', '小组', '小组赛', '抽签', '签在', '莫斯科', '科克', '克里', '克里姆', '克里姆林', '克里姆林宫', '里姆', '里姆林', '姆林宫', '举行']
['2018', '年', '世界', '世界杯', '小组', '小组赛', '抽签', '在', '莫斯科', '克里', '里姆', '克里姆', '里姆林', '姆林宫', '克里姆林宫', '举行']


## 6.2.2 词性标注

In [4]:
# 例6-2：显示例6-1分词后每个词的词性。
import jieba.posseg as pseg
words = pseg.cut("2018年世界杯小组赛抽签在莫斯科克里姆林宫举行")
for word, tag in words:
    print('word:{}, tag:{}'.format(word, tag) )

word:2018, tag:m
word:年, tag:m
word:世界杯, tag:nz
word:小组赛, tag:n
word:抽签, tag:v
word:在, tag:p
word:莫斯科, tag:nr
word:克里姆林宫, tag:nrt
word:举行, tag:v


## 6.2.3 特征提取

### 1.词袋模型

In [5]:
#例6-3：提取文档集的词袋特征

from sklearn.feature_extraction.text import CountVectorizer
import jieba
#给出文档集，放在字符串列表中
corpus = ["我是中国人，我爱中国", "我是上海人", "我住在上海松江大学城" ]
split_corpus = [] #初始化存储jieba分词后的列表
#循环为corpus中的每个字符串分词
for c in corpus:
    #将jieba分词后的字符串列表拼接为一个字符串，元素之间用" "分割
    s = " ".join(jieba.lcut(c)) #将分词得到的列表
    split_corpus.append(s) #将分词结果字符串添加到列表中
print(split_corpus)

['我 是 中国 人 ， 我 爱 中国', '我 是 上海 人', '我 住 在 上海 松江 大学城']


In [6]:
#生成词袋
cv = CountVectorizer()
cv_fit=cv.fit_transform(split_corpus)
print(cv.get_feature_names())  #显示特征列表
print(cv_fit.toarray())  #显示特征向量

['上海', '中国', '大学城', '松江']
[[0 2 0 0]
 [1 0 0 0]
 [1 0 1 1]]


In [7]:
#修改token_pattern默认参数,保留所有词特征
cv = CountVectorizer(token_pattern=r"(?u)\b\w+\b")
cv_fit=cv.fit_transform(split_corpus)
print(cv.get_feature_names())  #显示特征列表
print(cv_fit.toarray())  #显示特征向量

['上海', '中国', '人', '住', '在', '大学城', '我', '是', '松江', '爱']
[[0 2 1 0 0 0 2 1 0 1]
 [1 0 1 0 0 0 1 1 0 0]
 [1 0 0 1 1 1 1 0 1 0]]


### 2. TF-IDF模型

In [8]:
#例6-4：提取TF-IDF模型特征。

#第一种方法：将词袋特征转化为TF-IDF特征
from sklearn.feature_extraction.text import TfidfTransformer  
tfidf_transformer = TfidfTransformer()
tfidf_fit = tfidf_transformer.fit_transform(cv_fit)
#显示TF-IDF特征向量
print(tfidf_fit.toarray())

[[0.         0.72777291 0.27674503 0.         0.         0.
  0.42983441 0.27674503 0.         0.36388646]
 [0.52682017 0.         0.52682017 0.         0.         0.
  0.40912286 0.52682017 0.         0.        ]
 [0.34261996 0.         0.         0.45050407 0.45050407 0.45050407
  0.26607496 0.         0.45050407 0.        ]]


In [9]:
#第二种方法：直接使用feature_extraction.text模块的TfidfVectorizer类
from sklearn.feature_extraction.text import TfidfVectorizer
#直接用分词后得到的列表计算TF-IDF特征表示
tfidf = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b")
tfidf_fit=tfidf.fit_transform(split_corpus)
print(tfidf_fit.toarray()) #显示TF-IDF特征向量

[[0.         0.72777291 0.27674503 0.         0.         0.
  0.42983441 0.27674503 0.         0.36388646]
 [0.52682017 0.         0.52682017 0.         0.         0.
  0.40912286 0.52682017 0.         0.        ]
 [0.34261996 0.         0.         0.45050407 0.45050407 0.45050407
  0.26607496 0.         0.45050407 0.        ]]


# 6.3 垃圾邮件识别

## 6.3.2 构建文本分类特征训练集

In [2]:
#实例 垃圾邮件识别

from sklearn.feature_extraction.text import CountVectorizer
import jieba
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn import model_selection
from sklearn import metrics

#构造文本分类特征集
train_file = open("data/mailcorpus.txt", 'r', encoding = "utf-8")
corpus = train_file.readlines()   #列表中的每个元素为一行文本
#分词
split_corpus = []
for c in corpus:
    split_corpus.append(" ".join(jieba.lcut(c)))
    
#使用词袋模型提取特征，得到文本特征矩阵
cv = CountVectorizer(token_pattern=r"(?u)\b\w+\b")
X = cv.fit_transform(split_corpus).toarray()

#构造标签向量，垃圾标签为0，正常标签为1
y = [0] * 5000 + [1] * 5000

## 6.3.3 模型训练与验证

In [3]:
#将特征集 分为训练集和测试集
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.4, random_state = 0)

In [4]:
#使用朴素贝叶斯训练分类模型，给出分类效果
gnb = GaussianNB()
gnb.fit(X_train,y_train)

y_pred = gnb.predict(X_test) 

#朴素贝叶斯模型分类效果
print("naive_bayes accuracy:\n",gnb.score(X_test, y_test))
print("naive_bayes report:\n",metrics.classification_report(y_test, y_pred))
print("naive_bayes matrix:\n",metrics.confusion_matrix(y_test, y_pred))

naive_bayes accuracy:
 0.9865
naive_bayes report:
               precision    recall  f1-score   support

           0       1.00      0.98      0.99      1993
           1       0.98      1.00      0.99      2007

    accuracy                           0.99      4000
   macro avg       0.99      0.99      0.99      4000
weighted avg       0.99      0.99      0.99      4000

naive_bayes matrix:
 [[1948   45]
 [   9 1998]]


In [5]:
#使用SVM训练分类模型
svm = svm.SVC(kernel='rbf', gamma=0.7, C = 1.0)
svm.fit(X_train, y_train)

#SVM分类性能
y_pred_svm = svm.predict(X_test)   

print("SVM accuracy:\n",svm.score(X_test, y_test))
print("SVM report:\n",metrics.classification_report(y_test, y_pred_svm))
print("SVM matrix:\n",metrics.confusion_matrix(y_test, y_pred_svm))

SVM accuracy:
 0.92525
SVM report:
               precision    recall  f1-score   support

           0       1.00      0.85      0.92      1993
           1       0.87      1.00      0.93      2007

    accuracy                           0.93      4000
   macro avg       0.93      0.92      0.92      4000
weighted avg       0.93      0.93      0.92      4000

SVM matrix:
 [[1696  297]
 [   2 2005]]
